# Документация к проекту: Анализ продаж товаров Flipkart

**Цель проекта:** выявить ключевые закономерности в продажах товаров Flipkart и сделать рекомендации для бизнеса.  

**Дата:** январь 2026  
  
**Выполнила:** Привалова Анастасия 


## 1. Данные 

### Данные о товарах Flipkart

**Источник данных:** `flipkart.csv`  
**Размерность:** 80 000 строк, 25 колонок  

### Проверка данных и отчистка
- Пропущенные значения: только `size` (заполнены `Unknown`)  
- Дубликатов нет  
- Все столбцы приведены к корректным типам данных  
- Диапазоны значений логически согласованы  
- Уникальность идентификаторов подтверждена

### Таблица колонок
| Колонка             | Тип данных       | Описание |
|--------------------|----------------|----------|
| product_id          | object         | уникальный идентификатор товара |
| product_name        | object         | название товара |
| category            | object         | категория товара |
| brand               | object         | бренд |
| seller              | object         | продавец |
| seller_city         | object         | город продавца |
| price               | float64        | базовая цена |
| discount_percent    | int64          | процент скидки |
| final_price         | float64        | цена после скидки |
| rating              | float64        | средний рейтинг товара |
| review_count        | int64          | количество отзывов |
| stock_available     | int64          | количество на складе |
| units_sold          | int64          | проданные единицы |
| listing_date        | datetime64[ns] | дата размещения товара |
| size                | object         | размер, пропуски заполнены `Unknown` |
| ...                 | ...            | ... |

## 2. Exploratory Data Analysis (EDA)

**Цель:** исследовать распределения, взаимосвязи и закономерности в данных о товарах Flipkart  
**Источник данных:** `data/flipkart_cleaned.csv`

### 2.1 Распределение числовых признаков

Цель этапа — понять форму распределений, наличие асимметрии, выбросов и реалистичность значений.

<img src="../images/distribution/price_histplot.png" height="300"> <img src="../images/distribution/price_boxplot.png" height="300">

<img src="../images/distribution/discount_percent_histplot.png" height="300"> <img src="../images/distribution/discount_percent_boxplot.png" height="300">

<img src="../images/distribution/final_price_histplot.png" height="300"> <img src="../images/distribution/final_price_boxplot.png" height="300">

<img src="../images/distribution/rating_histplot.png" height="300"> <img src="../images/distribution/rating_boxplot.png" height="300">

<img src="../images/distribution/return_policy_days_histplot.png" height="300"> <img src="../images/distribution/return_policy_days_boxplot.png" height="300">

<img src="../images/distribution/shipping_weight_g_histplot.png" height="300"> <img src="../images/distribution/shipping_weight_g_boxplot.png" height="300">

#### Выводы по распределениям

- **Цена:** распределена равномерно в широком диапазоне, без выраженных выбросов.  
- **Скидка:** принимает дискретные значения (0%, 10%, 20%, 30%, 40%, 50%), медиана около 20%.  
- **Финальная цена:** правосторонняя асимметрия — дешёвых товаров больше.  
- **Рейтинг:** сосредоточен вокруг значения 3, без аномалий.  
- **Политика возврата:** дискретные значения, медиана 10 дней.  
- **Вес доставки:** правосторонняя асимметрия — тяжёлые товары встречаются редко.

### 2.2 Корреляция между числовыми признаками

**Цель:** выявить линейные зависимости между метриками и возможные факторы влияния на продажи.

<img src="../images/correlation/features_cor.png" height="500">

**Вывод:**  
Большинство корреляций носят технический характер (например, между price и final_price).  
Существенных линейных зависимостей между ключевыми бизнес-показателями (цена, рейтинг, скидка, продажи) не выявлено, что указывает на сложную структуру спроса и влияние скрытых факторов.

### 2.3 Анализ категориальных признаков

<img src="../images/categories/category.png" height="300"> <img src="../images/categories/brand.png" height="300">  
<img src="../images/categories/is_returnable.png" height="300"> <img src="../images/categories/payment_modes.png" height="300">

**Вывод:**  
Категориальные признаки в целом распределены достаточно равномерно, без доминирующих категорий.  
Единственный ярко выраженный паттерн — большинство товаров доступны для возврата, что отражает бизнес-политику платформы.

### 2.4 Взаимодействие признаков

<img src="../images/categories/сategory_depend.png" height="300"> <img src="../images/categories/discount_depend.png" height="300"> 

**Вывод:** Существенных различий между категориями по продажам, цене, срокам доставки и гарантии не выявлено.
Скидки немного варьируются между категориями, но различия не носят принципиального характера.

Анализ взаимодействия признаков показал, что ключевые характеристики товаров (категория, скидка, логистика, рейтинг) **не демонстрируют выраженного влияния на продажи и выручку по отдельности**.  
Это указывает на сложную структуру спроса и вероятное влияние факторов, не представленных в данных.

### 2.5 Анализ выручки и ключевых метрик

Создан признак отражающий размер выручки: `revenue_m` = final_price * units_sold / 1_000_000

<img src="../images/categories/top_3.png" height="300"> 

**Вывод:**  
Топ-3 категории формируют значимую долю выручки, однако доминирование не критично.  
Структура дохода диверсифицирована.

<img src="../images/correlation/rating_depend.png" height="300"> <img src="../images/correlation/discount_depend.png" height="300"> 

**Вывод:** Рейтинг не демонстрирует выраженной связи с выручкой.  
Наиболее высокие значения выручки наблюдаются у товаров без скидки, что может указывать на слабую ценовую эластичность спроса.

### 2.6 Итог EDA и гипотезы

#### Ключевые наблюдения:

- **Цены и скидки**: 
  - Цены распределены равномерно в широком диапазоне, без выраженной асимметрии и выбросов, что отражает разнообразие товаров — от бюджетных до премиальных.  
  - Скидки имеют дискретные уровни (0%, 10%, 20%, 30%, 40%, 50%) и сосредоточены в диапазоне 10–40%, медиана около 20%. Финальная цена со скидкой демонстрирует правостороннюю асимметрию: дешёвых товаров больше, дорогие встречаются реже.

- **Рейтинг товаров**: распределён симметрично вокруг медианы 3, с частыми значениями, но без выраженных выбросов.

- **Политика возврата**: представлена дискретными значениями, встречающимися примерно одинаково часто. Медиана — 10 дней, редкий выброс на 30 дней отражает редкие длительные варианты возврата.

- **Вес при транспортировке**: распределение с правосторонней асимметрией — лёгкие и средние товары встречаются чаще, тяжёлые — реже.

- **Категориальные признаки**: в целом сбалансированы, без доминирующих категорий. Основной паттерн — преобладание возможности возврата товара, что отражает бизнес-политику платформы. Существенных зависимостей между категорией товара и ключевыми метриками (**цена, продажи, срок доставки, размер гарантии**) не выявлено.

- **Взаимосвязь признаков**:  
  - Линейные зависимости между ценой, скидкой, рейтингом и продажами отсутствуют.  
  - Межквартильный размах скидок у категорий примерно одинаковый, медианы 15–20%.  
  - Выручка распределена умеренно равномерно по категориям; топ-3 категории — основные драйверы дохода, но их доминирование не критично.  
  - Scatterplot и boxplot показывают слабое влияние рейтинга и скидки на выручку, а наибольшие значения выручки наблюдаются у товаров без скидки, что указывает на слабую ценовую эластичность спроса.

- **Выбросы и ключевые метрики**: явных выбросов нет, за исключением редких тяжёлых товаров и длительных вариантов возврата. Разброс выручки присутствует при любом уровне рейтинга, что отражает влияние множества факторов на продажи.

#### Вывод

Распределение ключевых признаков соответствует бизнес-логике: широкий ценовой диапазон, типичные уровни скидок, умеренно равномерная выручка, редкие тяжёлые товары и длительные варианты возврата.  

Существенных линейных зависимостей между основными метриками не выявлено, что указывает на сложную структуру спроса и влияние скрытых факторов (сезонность, маркетинговые акции, ассортиментная стратегия).  

#### Гипотезы

1. **Ценовой сегмент и выручка:** медианная выручка увеличивается с повышением ценового сегмента (Low < Mid < High), что указывает на зависимость продаж от ценовой категории товара.
  
2. **Влияние рейтинга на продажи:** рейтинг товара не оказывает значимого линейного влияния на выручку, однако возможны скрытые эффекты, связанные с ценовым сегментом или сезонностью.
  
3. **Сезонность продаж:** продажи демонстрируют повторяющиеся пики в определённые месяцы, что может отражать сезонный спрос.

4. **Влияние быстрой доставки:** товары с коротким сроком доставки или более быстрой логистикой могут показывать более высокую выручку, особенно в пиковые сезоны.

## 3. Углубленный анализ (Feature Engineering)
  
**Цель:** создать дополнительные бизнес-признаки и проверить гипотезы о факторах, влияющих на выручку и продажи.  
**Источник данных:** `data/flipkard_after_EDA.csv`

**Проверяемые гипотезы:**  
- выручка растет с увеличением ценового сегмента
- рейтинг не оказывает существенного линейного влияния на продажи
- продажи имеют сезонный характер 
- быстрая доставка может положительно влиять на выручку

### 3.1 Созданные технические признаки 

- Ценовые и рейтинговые сегменты: low / mid / high  
- Временные признаки: месяц, год размещения  
- Логистические признаки (быстрая доставка)
- Признаки доверия и агрегированные признаки 
- Метрики эффективности:
  - discount_amount и discount_ratio (абсолютная и относительная скидка)
  - sell_through_rate (доля проданных товаров)

### 3.2 Результаты углубленного анализа

#### Выручка по ценовым сегментам 
  
<img src="../images/segments/price_segment.png" height="300"> 

**Вывод:**  
Медианная выручка растёт от Low к High сегменту. Ценовой сегмент является значимым фактором для выручки.

#### Рейтинг и продажи

<img src="../images/segments/rating_segment.png" height="300"> 

**Вывод:**
Различий между сегментами не выявлено. Рейтинг не оказывает заметного влияния на продажи.

#### Сезонность

<img src="../images/seasons/general.png" height="300"> <img src="../images/seasons/all_years.png" height="300"> 

При анализе сезонности было выявлено, что не все годы в датасете содержат данные за полный календарный период (12 месяцев).  
Использование таких лет в расчётах искажает сезонный паттерн и может создавать ложное ощущение падения спроса во второй половине года.

<img src="../images/seasons/general_full.png" height="300"> 

Поэтому для корректного анализа сезонности дополнительно был построен график только по полным годам.  
После исключения неполных периодов сезонный паттерн стал более устойчивым.

**Вывод:**  
Обнаружены устойчивые пики продаж в январе, марте, мае, июле и ноябре. Подтверждена сезонность спроса.

#### Быстрая доставка

<img src="../images/segments/fast_delivery.png" height="300"> 

**Вывод:**
Различия в медианной выручке минимальны. Существенного эффекта скорости доставки не выявлено.

### 3.3 Итог углубленного анализа и гипотезы

1. **Влияние ценового сегмента:**  
   Медианная выручка растёт с увеличением ценового сегмента. Это подтверждает, что ценовой сегмент является важным фактором влияния на продажи. Для увеличения дохода рекомендуется фокусироваться на High-сегменте, при этом Low и Mid обеспечивают стабильный поток продаж.

2. **Влияние рейтинга:**  
   Медианные значения выручки практически одинаковы для всех сегментов рейтинга. Это свидетельствует о том, что рейтинг товара **не оказывает значимого линейного влияния на выручку**.

3. **Сезонность продаж:**  
   Анализ выручки по месяцам выявил повторяющиеся локальные пики: январь, март, май, июль, август и ноябрь. Наиболее заметный спад наблюдается в феврале, что формирует характерный «провал» после январского максимума. Это подтверждает гипотезу о **сезонности продаж** и указывает на цикличность спроса в течение года.

4. **Влияние скорости доставки:**  
   Медианная выручка у товаров с быстрой доставкой немного выше, чем у товаров с обычной доставкой, однако разница незначительная. Это указывает на **отсутствие заметного влияния скорости доставки на медианную выручку**. Возможны скрытые эффекты других факторов (цена, категория товара, сезонный спрос), которые потребуют дальнейшей проверки на этапе моделирования.


**Вывод:**  
Данные этапа углубленного анализа показали, что ключевыми факторами, влияющими на выручку, являются **ценовой сегмент и сезонность**. Эти наблюдения формируют основу для дальнейшей проверки гипотез на этапе моделирования и позволяют выбрать информативные признаки для прогнозных моделей.

#### Гипотезы

1. **Гипотеза 1 (сезонность)**:  
После январского пика наблюдается наиболее резкий спад в феврале,   формирующий выраженный сезонный «провал». Выравнивание этого снижения может способствовать повышению годовой выручки.

2. **Гипотеза 2 (скорость доставки)**:  
Скорость доставки оказывает слабое влияние на выручку, либо её эффект скрыт другими факторами, такими как ценовой сегмент, категория товара или сезонный спрос. Для точного подтверждения требуется проведение статистической проверки.

## 4. Моделирование 
  
**Цель:** Статистически проверить ключевые гипотезы и определить факторы, оказывающие влияние на выручку.
**Данные:** `flipkard_with_features.csv`

### 4.1 Проверка гипотезы о сезонности

**Метод:** однофакторный ANOVA по месяцам  
  
**Результат:**
p-value > 0.05 – статистически значимых различий между месяцами не выявлено.
Дополнительно проверена гипотеза о «февральском спаде» (непараметрический тест (Mann–Whitney) между январем и февралем) – различия также не являются статистически значимыми.  
  
**Вывод:**
Визуально наблюдаемая сезонность на суммарных графиках не подтверждается статистически.
Сезонные колебания объясняются структурой данных (количество объектов, длительность месяца), а не реальными изменениями средней выручки.

### 4.2 Проверка гипотезы о влиянии скорости доставки 

**Метод:** множественная линейная регрессия (OLS, statsmodels)  
  
**Признаки:** ценовой сегмент, рейтинг, месяц размещения, скорость доставки
  
**Результаты модели:**
Ценовой сегмент — статистически значимый фактор
**Скорость доставки — незначимый фактор (p-value = 0.716)**
Рейтинг — незначимый фактор
R² ≈ 0.35
  
**Вывод:**
Выручка в первую очередь определяется ценовым сегментом товара, а не логистикой или рейтингом.

### 4.3 Оценка качества модели

**Метрики:**
R² ≈ 0.34
MAE ≈ 33
  
**Интерпретация:**
Модель не предназначена для точного прогноза, но хорошо подходит для аналитической интерпретации факторов влияния, что соответствует цели проекта.

### 4.4 Итог моделирования

**Вывод:**  
Статистическое моделирование подтвердило, что ключевым фактором, влияющим на выручку, является ценовой сегмент товара. Сезонность, рейтинг и скорость доставки не демонстрируют статистически значимого влияния на среднюю выручку.Приоритетные направления для оптимизации дохода — корректировка ассортимента и ценовой политики.
  
Модель объясняет около 35% вариации выручки и может использоваться как интерпретируемый инструмент для анализа бизнес-факторов.


## 5. Симуляция A/B теста
  
**Цель:** проверить, как изменение цены (±5%) влияет на выручку и средний чек в однородном сегменте товаров.  
**Данные:** `flipkard_with_features.csv`
**Почему симуляция:** исторические данные, реальные изменения цены для клиентов невозможны, поэтому создаём случайные группы и моделируем изменение цены.

**Гипотезы:**  
H₀: изменение цены не влияет на выручку  
H₁: изменение цены влияет на выручку  
  
**Метрики:** revenue_m, units_sold, средний чек (avg_order)

### 5.1 Выбор сегмента товара и формирование групп

Для уменьшения влияния посторонних факторов выбран однородный сегмент:  
- средний ценовой сегмент (mid),
- высокий рейтинг (high),
- возвратный товар,
- надежный продавец.  
  
Сегмент позволяет сосредоточиться на эффекте цены без вмешательства других факторов.  
  
Случайное разделение на группы A (контроль) и B (тест, +5% к цене) обеспечивает корректное сравнение.  
Выручка и средний чек пересчитаны с учётом модифицированной цены

### 5.2 Визуальный анализ

<img src="../images/ab_test/revenue.png" height="300">

**Вывод:**  
- Разница между группами почти отсутствует.
- Повышение цены на 5% не изменило суммарную выручку существенно.

<img src="../images/ab_test/avg_order.png" height="300">

**Вывод:**  
- Средний чек у группы B выше, что ожидаемо, поскольку в данных цена = средний чек.
- Разница умеренная, распределения перекрываются.

### 5.3 Статистическая проверка

- t-тест для выручки: p-value = 0.233 (> 0.05)
- Разница средних выручек: ~3 млн
  
**Вывод:**  
Статистически значимого влияния повышения цены на выручку не выявлено. Нулевая гипотеза H₀ не отвергнута. Случайное распределение товаров и клиентов объясняет небольшие различия.

### 5.4 Итог A/B теста (симуляции)

- Разница средних выручек между группами: 3.00
- p-value: 0.233
- **Вывод:** статистически значимого влияния повышения цены на 5% на выручку не выявлено (H₀ не отвергнута). Распределения выручки по группам практически совпадают, что визуально подтверждает отсутствие значимого эффекта.
- **Особенности данных:** в данных цена товара фактически равна среднему чеку, поэтому анализ среднего чека отражает прямое изменение цены.
- **Ограничения симуляции:** снижение количества проданных единиц (реакция покупателей на повышение цены) не моделировалось, поэтому возможное влияние на выручку в реальных условиях может быть недооценено.
- **Рекомендация:** небольшие корректировки цен (±5%) можно проводить без риска значительного снижения выручки; повышение цены увеличивает средний чек, но общая выручка остаётся почти на том же уровне в рамках текущей симуляции.